# YOLO - Inferencia
## Detección de objetos

**Autor original:** @theAIGuysCode<br>
**Adaptado:** @maxiyommi<br>
**Fecha:** jun 2021<br>
**Descripción:** Notebook para inferir resultados.

## Instalación de OpenCV

``` bash
    sudo apt install python3-opencv
    sudo apt-get install libopencv-dev
    sudo apt install python3-venv
```

## Instalar zip

``` bash
    sudo apt install zip unzip
``` 

## Preparación del entorno

* Crear entorno virtual (solo la primera vez) dentro de la carpeta `/inference`.

``` bash
    python3 -m venv yolo_inference --system-site-packages # crea el entorno virtual 
```

* Activar entorno virtual (siempre).

``` bash
    source yolo_inference/bin/activate # activa el entorno virtual
    deactivate # desactiva el entorno virtual
    rm -r <nombre_del_entorno> # eliminar el entorno virtual
```

* Instalar librerias en el entorno virtual creado (solo la primera vez, a menos que se actualice).

``` bash
    pip3 install -r requirements.txt 
```

## Importando librerias

In [14]:
import sys

sys.path.append('../script')

import ibmCloud as ibm
from gen_txt_inference import gen_text

## Clonación y construcción de Darknet (por única vez)

In [ ]:
# clonar repositorio darknet
!git clone https://github.com/AlexeyAB/darknet

## Compilando Darknet

Modificar el archivo `/darknet/Makefile`. Una opción es abrirlo directamente o usando las siguientes instrucciones.

In [5]:
%cd darknet

[Errno 2] No such file or directory: 'darknet'
/home/max/Dropbox/Git/proyectos/UALI/ml_uali/inference/darknet


In [ ]:
# cambie el makefile para tener GPU y OPENCV habilitados (para pruebas locales)
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=1/GPU=0/' Makefile
!sed -i 's/CUDNN=1/CUDNN=0/' Makefile
!sed -i 's/CUDNN_HALF=1/CUDNN_HALF=0/' Makefile

Modificar las siguientes lineas para redireccionar el path de CUDA

```
GPU=1
CUDNN=1

ifeq ($(GPU), 1)
COMMON+= -DGPU -I/usr/local/cuda-11.4/include/
CFLAGS+= -DGPU
ifeq ($(OS),Darwin) #MAC
LDFLAGS+= -L/usr/local/cuda-11.4/lib -lcuda -lcudart -lcublas -lcurand
else
LDFLAGS+= -L/usr/local/cuda-11.4/lib64 -lcuda -lcudart -lcublas -lcurand
endif
endif

ifeq ($(CUDNN), 1)
COMMON+= -DCUDNN
ifeq ($(OS),Darwin) #MAC
CFLAGS+= -DCUDNN -I/usr/local/cuda-11.4/include
LDFLAGS+= -L/usr/local/cuda-11.4/lib -lcudnn
else
CFLAGS+= -DCUDNN -I/usr/local/cudnn/include
LDFLAGS+= -L/usr/local/cudnn/lib64 -lcudnn
endif
endif
```

> **Nota:** Tambien se puede usar el archivo `Makefile_mod` para reemplazar (recordar modificar el nombre)

In [ ]:
# make darknet (crea darknet para que luego pueda usar el archivo ejecutable darknet para ejecutar o entrenar detectores de objetos)
!make -h

In [ ]:
!make -d

In [ ]:
./darknet detector test ./cfg/coco.data ./cfg/yolov4.cfg ./yolov4.weights

## Descargando pesos de YOLOv4 (entrenamiento propio)

### Yolov4 - openImage (yolo_v1)

* weights-file (best): --> URL: https://drive.google.com/file/d/1-5eprW8D2Si3gZOqaN4QadHOFhvu6OWT/view?usp=sharing

* weights-file (final) --> URL: https://drive.google.com/file/d/1-00dw9YYg9uz1qFGxxJbmjSC_7C6yG_H/view?usp=sharing

* data_training (.cfg - .names - .data) --> URL: https://github.com/githubuali/ml_uali/blob/main/training/yolo_v1/data_training.zip?raw=true

### Yolov4 - visDrone (yolo_v2)

* weights-file (best): --> URL: https://drive.google.com/file/d/1Joo7P5zjiqnJuJr4KfouIpNZshnvLPYd/view?usp=sharing

* data_training (.cfg - .names - .data) --> URL: https://github.com/githubuali/ml_uali/blob/main/training/yolo_v2/data_training.zip?raw=true

### Download

In [ ]:
# descargar weights
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Joo7P5zjiqnJuJr4KfouIpNZshnvLPYd' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Joo7P5zjiqnJuJr4KfouIpNZshnvLPYd" -O ../yolov4-obj_final.weights && rm -rf /tmp/cookies.txt

In [ ]:
# descargar comprimido de data_training
!wget -O ../data_training.zip https://github.com/githubuali/ml_uali/blob/main/training/yolo_v2/data_training.zip?raw=true

### Ordenando archivos

In [ ]:
# descomprima los conjuntos de datos 
!unzip ../data_training.zip -d ../data_training

In [ ]:
# copiar obj.name
!cp ../data_training/data_training/obj.names data

In [ ]:
# eliminando carpetas
!rm -rf ../data_training.zip

### Ejecutando Darknet y YOLOv4

### Por URL

URL: https://drive.google.com/file/d/1aLTw006n7dTYk_9P8TsY7tN-3xd-txLW/view?usp=sharing

In [ ]:
# descargar comprimido de imagenes de prueba
!wget -O ../image_ypf.zip https://github.com/githubuali/ml_uali/blob/main/data/image_ypf.zip?raw=true

# descomprima los conjuntos de datos 
!unzip ../image_ypf.zip

# eliminando carpetas
!rm -rf ../image_ypf.zip

In [26]:
download_path ='../eventos/eventosDetectados_YPF'

### Por IBM Cloud 

In [12]:
prefix = '28072021_SN_'
download_path = '../eventos/' +  prefix

In [ ]:
cos = ibm.configInstance()
buckets = ibm.getBuckets(cos)   
keys = ibm.getObjectsNames(cos, buckets[0], prefix) 
ibm.downloadFiles(cos, buckets[0], keys[0::], download_path)

### Generar el .txt con paths

In [27]:
gen_text (download_path+'/')

In [28]:
%cd darknet/

[Errno 2] No such file or directory: 'darknet/'
/home/max/Dropbox/Git/proyectos/UALI/ml_uali/inference/darknet


In [ ]:
# Necesitamos configurar nuestro cfg personalizado en modo de prueba (unica vez)
!sed -i 's/batch=64/batch=1/' ../data_training/data_training/yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' ../data_training/data_training/yolov4-obj.cfg

In [ ]:
import time
inicio = time.time()

time.strftime('La inferencia se inició: '+'%Y-%m-%d %H:%M:%S', time.localtime())

In [ ]:
#ejecute su detector personalizado con este comando (cargue una imagen en su unidad de Google para probar, la bandera thresh establece la precisión que debe tener la detección para mostrarla)
!./darknet detector test ../data_training/obj.data ../data_training/yolov4-obj.cfg ../yolov4-obj_final.weights -ext_output -dont_show -out result.json < ../eventos/eventosDetectados_YPF/images.txt

time.strftime('La inferencia finalizó: '+'%Y-%m-%d %H:%M:%S', time.localtime())

fin = time.time()
print('El tiempo total de proceso es de: ' + str((fin-inicio)*(1.7*(10**-5)))+ ' minutos.') 